In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [2]:
from drone_sar.ipsar_dataset import IPSARDataset
from drone_sar.lightning_detector import get_lightning_trainer, LightningDetector
from collections import Counter
from drone_sar.vis import show_detections

In [3]:
train_data = IPSARDataset("../heridal/trainImages/")
test_data = IPSARDataset("../heridal/testImages")
item = test_data[0]

In [4]:
trainer = get_lightning_trainer("IPSAR-DETR-FIXED", max_epochs=1000)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
# best_model_path = "/home/iz/workspace/DroneSearchAndRescue/notebooks/.checkpoints/epoch=88-step=14151.ckpt"
best_model_path = "/home/iz/workspace/DroneSearchAndRescue/notebooks/.checkpoints/epoch=65-step=10494.ckpt"

In [6]:
best_model = LightningDetector.load_from_checkpoint(best_model_path, lr=0.01)

Some weights of DetrForObjectDetection were not initialized from the model checkpoint at facebook/detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([92, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
test_dl_b1 = test_data.get_dataloader(best_model.processor, bs=1, shuffle=False)
train_dl_b1 = train_data.get_dataloader(best_model.processor, bs=1, shuffle=False)

In [8]:
best_val_outputs = trainer.predict(best_model, test_dl_b1)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Predicting: |                                                      | 0/? [00:00<?, ?it/s]

In [11]:
best_train_outputs = trainer.predict(best_model, train_dl_b1)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                      | 0/? [00:00<?, ?it/s]

In [12]:
best_train_outputs[0].keys()

odict_keys(['loss', 'loss_dict', 'logits', 'pred_boxes', 'last_hidden_state', 'encoder_last_hidden_state'])

In [9]:
pil = test_data[1]["pil"]

In [25]:
# boxes = best_model.predict(pil)

In [26]:
# plt_show(pil, colored_boxes_dict={"red": boxes})

In [11]:
import cv2
from PIL import Image
from tqdm.auto import tqdm
import numpy as np

In [12]:
vid = cv2.VideoCapture("../DJI_0004.MP4")

fps = np.ceil(vid.get(cv2.CAP_PROP_FPS)).astype(int)
num_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames, fps

(8722, 30)

In [14]:
# best_model.predict(pil)

In [17]:
pils = []
preds = []

for i in tqdm(range(num_frames)):
    success, frame = vid.read()
    if not success:
        break
    if i % 20 == 0:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil = Image.fromarray(frame)
        best_val_outputs = best_model.predict(pil)

        pils.append(pil)
        preds.append(best_val_outputs)

  0%|          | 0/8722 [00:00<?, ?it/s]

In [18]:
def draw_res(pil, pred):
    im = np.array(pil)

    for x0, y0, w, h in pred:
        x1, y1 = x0 + w, y0 + h
        x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)

        im = cv2.rectangle(im, (x0, y0), (x1, y1), (255, 0, 0), 5)
    im = Image.fromarray(im)
    im.thumbnail((2000, 2000))

    return im

In [19]:
rendered_frames = []
for pil, pred in tqdm(list(zip(pils, preds))):
    rend = draw_res(pil, pred)
    rendered_frames.append(rend)

  0%|          | 0/430 [00:00<?, ?it/s]

In [20]:
len(rendered_frames)

430

In [21]:
rendered_frames[0].size

(2000, 1125)

In [22]:
# rendered_frames[0]

In [23]:
width, height = rendered_frames[0].size
result = cv2.VideoWriter(
    "red2.avi",
    cv2.VideoWriter_fourcc(*"MJPG"), 
    20,
    (width, height),
)

In [24]:
for image in tqdm(rendered_frames):
    im = np.array(image)
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    result.write(im)

result.release()

  0%|          | 0/430 [00:00<?, ?it/s]

In [318]:
# from IPython.display import Video

# Video("red.avi")